In [1]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta

from bulletin.data.notifica import Notifica
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer

pd.set_option('display.max_columns', None)

output = join("output","correcoes","notifica")

if not isdir(output):
    makedirs(output)

In [2]:
timer = Timer()
notifica = Notifica()
# notifica.update()
print(timer.stop())
notifica.shape()

426.3553116321564


(1709432, 755234, 7044, 945478, 1676)

In [3]:
timer.reset()
casos_confirmados = CasosConfirmados()
# casos_confirmados.update()
print(timer.stop())
casos_confirmados.shape()

0.7634501457214355


(370775, 7213)

In [4]:
casosn = notifica.get_casos()
casosc = casos_confirmados.get_casos()
casosn.groupby('classificacao_final')[['id']].count()

,id
classificacao_final,
CASO CONFIRMADO,368187
CASO DESCARTADO,565711
CASO SUSPEITO,767807
IGNORADO,700
SINDROME GRIPAL NAO ESPECIFICADA,7027


In [5]:
def get_better(df):
    scores = np.zeros(len(df))

    for i, serie in enumerate(df.iterrows()):
        _, row = serie

        if row['data_nascimento'] != pd.NaT:
            scores[i]+=50

        if row['nome_mae']:
            scores[i]+=50

        if row['cpf']:
            scores[i]+=50

        if not row['nome_mae'] and not row['cpf'] and row['data_nascimento'] == pd.NaT:
            scores-=500

        if row['cod_classificacao_final'] == 2:
            scores[i]+=100

        if row['cod_criterio_classificacao'] == 1:
            scores[i]+=10

        if row['cod_evolucao'] == 2:
            scores[i]+=1000

        if row['cod_metodo'] == 1:
            scores[i]+=10

        if row['cod_status_notificacao'] in [1, 2]:
            scores[i]+=10

        if row['excluir_ficha'] == 2:
            scores[i]+=10

        if row['cod_origem'] == 1:
            scores[i]+=10

    i = np.argmax(scores)
    return df.iloc[i].name

In [6]:
casos_duplicados_cpf = casosn.loc[(casosn['cpf'].notnull()) & (casosn.duplicated('cpf',keep=False))].sort_values('cpf')
print(f"Total de pacientes com mais de uma ocorrencia: {len(casos_duplicados_cpf)}")
print(f"{len(casos_duplicados_cpf.groupby('cpf'))} pacientes pelo CPF que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados_cpf.groupby('cpf'):
    idx = get_better(group)
    manter.append(idx)


casos_duplicados_cpf = set(casos_duplicados_cpf.index.tolist())
casos_duplicados_cpf = casos_duplicados_cpf - set(manter)
casosn.loc[casos_duplicados_cpf].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia: 317579
130654 pacientes pelo CPF que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,8431
CASO DESCARTADO,69079
CASO SUSPEITO,108326
IGNORADO,47
SINDROME GRIPAL NAO ESPECIFICADA,1042


In [7]:
casosn = casosn.drop(index=casos_duplicados_cpf)
casosn.loc[manter].groupby('classificacao_final')[['id']].count()

,id
classificacao_final,
CASO CONFIRMADO,42620
CASO DESCARTADO,63577
CASO SUSPEITO,23777
IGNORADO,9
SINDROME GRIPAL NAO ESPECIFICADA,671
